# Getting started with INTEGRATE

This notebook contains a simple example of geeting started with INTEGRATE

In [ ]:
import integrate as ig


In [ ]:
#f_data_h5 = 'tTEM_20230727_20230814_RAW_export.h5'
#f_data_h5 = 'tTEM_20230727_20230814_AVG_export_J1000.h5'
f_data_h5 = 'tTEM_20230727_20230814_AVG_export_J200.h5'
id = 1  
file_gex= ig.get_gex_file_from_data(f_data_h5, id=id)

print("Using GEX file: %s" % file_gex)

## 1. Setup the prior model ($\rho(\mathbf{m},\mathbf{d})$

In this example a simple layered prior model will be considered

### 1a. first, a sample of the prior model parameters, $\rho(\mathbf{m})$, will be generated

In [ ]:
N=500000
f_prior_h5 = ig.prior_model_layered(N=N,lay_dist='chi2', NLAY_deg=5)


### 1b. Then, a corresponding sample of $\rho(\mathbf{d})$, will be generated

In [ ]:
f_prior_data_h5 = ig.prior_data_gaaem(f_prior_h5, file_gex)


## Sample the posteriorm $\sigma(\mathbf{m})$

The posterior distribution is sampling using the extended rejection sampler.

f_post_h5 = ig.integrate_rejection(f_prior_data_h5, f_data_h5, N_use = 50000000, parallel=1, updatePostStat=False, showInfo=1)
# Compute some generic statistic of the posterior distribtiuon (Mean, Median, Std)
ig.integrate_posterior_stats(f_post_h5)

### Plot some statistic from $\sigma(\mathbf{m})$

In [ ]:
# Plot the Temperature used for inversion
ig.plot_T(f_post_h5)

In [ ]:
ig.plot_profile_continuous(f_post_h5, i1=1000, i2=2000, im=1)

In [ ]:

# Plot a 2D feature: The number of layers
ig.plot_feature_2d(f_post_h5,im=2,iz=0,key='Median', title_text = 'Number of layers', cmap='jet', s=12)

# Plot a 2D feature: Resistivity in layer 10
ig.plot_feature_2d(f_post_h5,im=1,key='Median', uselog=1, cmap='jet', s=10)
#ig.plot_feature_2d(f_post_h5,im=1,iz=80,key='Median')